In [1]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [2]:

from __future__ import print_function

import re
import sys
from operator import add

from pyspark.sql import SparkSession


In [3]:
lines = sc.textFile('data/pagerank_data.txt')
# lines.collect()

In [4]:
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
# .mapValues(list).collect()
# links

In [5]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [6]:
for iteration in range(int(2)):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

In [7]:
for (link, rank) in ranks.collect():
    print("%s has rank: %s." % (link, rank))

2 has rank: 0.915.
4 has rank: 0.915.
1 has rank: 1.255.
3 has rank: 0.915.


In [8]:
# ranks.collect()